# Affect Reconition

In [2]:
import cv2
import opencv_jupyter_ui as jcv2
from feat import Detector
from IPython.display import Image


from feat.utils import FEAT_EMOTION_COLUMNS
from feat.pretrained import AU_LANDMARK_MAP

import pandas as pd

In [3]:
detector = Detector(device="cuda")

INFO:root:Loading Face model: retinaface
g:\My Drive\Uppsala\IIS\venv\Lib\site-packages\feat\face_detectors\Retinaface\Retinaface_test.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimen

In [4]:
import torch
import numpy as np
# emotion read as string
df = pd.read_csv("dataset.csv", dtype={"emotion": str})



# get the list of emotions
emotions = df['emotion'].unique()

# encode the emotions as integers
emotion_to_int = {emotion: i for i, emotion in enumerate(emotions)}

# features: AU01,AU02,AU04,AU05,AU06,AU07,AU09,AU10,AU11,AU12,AU14,AU15,AU17,AU20,AU23,AU24,AU25,AU26,AU28,AU43
# labels: emotion
features = df[AU_LANDMARK_MAP["Feat"]].values
labels = df['emotion'].map(emotion_to_int).values

# convert to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Train/val/test split percentage: 70/20/10
train_size = int(0.7 * len(df))
val_size = int(0.2 * len(df))
test_size = len(df) - train_size - val_size

# split the data
train_features, val_features, test_features = features[:train_size], features[train_size:train_size+val_size], features[train_size+val_size:]
train_labels, val_labels, test_labels = labels[:train_size], labels[train_size:train_size+val_size], labels[train_size+val_size:]



In [5]:
print("Train features shape:", train_features.shape)
print("Train labels shape:", train_labels.shape)
print("Val features shape:", val_features.shape)
print("Val labels shape:", val_labels.shape)
print("Test features shape:", test_features.shape)
print("Test labels shape:", test_labels.shape)

Train features shape: (812, 20)
Train labels shape: (812,)
Val features shape: (232, 20)
Val labels shape: (232,)
Test features shape: (117, 20)
Test labels shape: (117,)


In [6]:
import sklearn
from sklearn.metrics import accuracy_score

## sklearn model: Support Vector Machines (SVM)

In [7]:
from sklearn.svm import SVC

# SVC with linear kernel
# train the model
model_SVC_linear = SVC(kernel='linear')
model_SVC_linear.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_SVC_linear.predict(val_features)

# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for SVC (kernel='linear'): {accuracy}")

Validation Accuracy for SVC (kernel='linear'): 0.5862068965517241


In [8]:
# SVC with RBF kernel
# train the model
model_SVC_rbf = SVC(kernel='rbf')
model_SVC_rbf.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_SVC_rbf.predict(val_features)

# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for SVC (kernel='rbf'): {accuracy}")


Validation Accuracy for SVC (kernel='rbf'): 0.5905172413793104


In [9]:
# SVC with poly kernel
# train the model
model_SVC_poly = SVC(kernel='poly')
model_SVC_poly.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_SVC_poly.predict(val_features)

# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for SVC (kernel='poly'): {accuracy}")

Validation Accuracy for SVC (kernel='poly'): 0.5862068965517241


In [10]:
# SVC with sigmoid kernel
# train the model
model_SVC_sigmoid = SVC(kernel='sigmoid')
model_SVC_sigmoid.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_SVC_sigmoid.predict(val_features)

# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for SVC (kernel='sigmoid'): {accuracy}")

Validation Accuracy for SVC (kernel='sigmoid'): 0.2025862068965517


In [11]:
# SVC with precomputed kernel
# train the model
model_SVC_precomputed = SVC(kernel='precomputed')
model_SVC_precomputed.fit(train_features @ train_features.T, train_labels)

# predict the validation set
val_predictions = model_SVC_precomputed.predict(val_features @ train_features.T)

# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for SVC (kernel='precomputed'): {accuracy}")


Validation Accuracy for SVC (kernel='precomputed'): 0.5862068965517241


In [12]:
# SVC test accuracy with rbf kernel
test_predictions = model_SVC_rbf.predict(test_features)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy for SVC (kernel='rbf'): {test_accuracy}")

Test Accuracy for SVC (kernel='rbf'): 0.5982905982905983


In [14]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

# KNN with k=1
# train the model

model_KNN_1 = KNeighborsClassifier(n_neighbors=1)
model_KNN_1.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_KNN_1.predict(val_features)


# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for KNN (k=1): {accuracy}")

# KNN with k=3
# train the model
model_KNN_3 = KNeighborsClassifier(n_neighbors=3)
model_KNN_3.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_KNN_3.predict(val_features)


# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for KNN (k=3): {accuracy}")


# KNN with k=5
# train the model
model_KNN_5 = KNeighborsClassifier(n_neighbors=5)
model_KNN_5.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_KNN_5.predict(val_features)


# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for KNN (k=5): {accuracy}")

# KNN with k=7
# train the model
model_KNN_7 = KNeighborsClassifier(n_neighbors=7)
model_KNN_7.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_KNN_7.predict(val_features)



# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for KNN (k=7): {accuracy}")
# KNN with k=9
# train the model
model_KNN_9 = KNeighborsClassifier(n_neighbors=9)
model_KNN_9.fit(train_features, train_labels)

# predict the validation set
val_predictions = model_KNN_9.predict(val_features)


# calculate the accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy for KNN (k=9): {accuracy}")








Validation Accuracy for KNN (k=1): 0.5387931034482759
Validation Accuracy for KNN (k=3): 0.5517241379310345
Validation Accuracy for KNN (k=5): 0.5301724137931034
Validation Accuracy for KNN (k=7): 0.5646551724137931
Validation Accuracy for KNN (k=9): 0.5301724137931034


In [ ]:
#TEST
# KNN test accuracy with k=7

test_predictions = model_KNN_7.predict(test_features)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy for KNN (k=7): {test_accuracy}")




Test Accuracy for KNN (k=7): 0.5897435897435898
